<div style="text-align:center; margin:auto">
<h1>Ischemic Stroke Visualization Dashboard</h1>
</div>

<div style="text-align:center; margin:auto">
<img src='./brainclot.jpg' height=400px width=600px>
</div>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from ipywidgets import interact

In [2]:
RED = '#CA0016'
BLUE = '#009797'
OCHRE = '#E0D897'

In [3]:
df = pd.read_csv('stroke.csv')
df.head()

,Patient Id,Age,Gender,Comorbidities,Emergency Dept Time,Admission Time,Discharge Time,CT Scan Time,TPA Time,ICU Arrival Time,ICU Checkout Time,Neurology Ward Arrival Time,Occupational Therapist Visit,Speech Pathologist Visit,Physiotherapist Visit,Dietitian Visit,Social Worker Visit,Cardiologist Visit,Neurologist Visit
0,0,67,F,True,2022-06-15 19:27:00,2022-06-16 03:57:00,2022-07-04 07:07:00,2022-06-15 19:45:00,NaN,NaN,NaN,2022-06-16 05:01:00,2022-06-18 23:28:00,2022-07-04 06:26:00,2022-06-29 06:35:00,NaN,NaN,NaN,2022-06-20 06:47:00
1,1,75,F,False,2023-01-04 23:20:00,2023-01-05 02:13:00,2023-01-10 10:59:00,2023-01-04 23:33:00,NaN,2023-01-05 02:14:00,2023-01-06 09:44:00,2023-01-06 09:44:00,NaN,2023-01-07 01:21:00,2023-01-05 03:13:00,NaN,2023-01-06 00:39:00,NaN,2023-01-06 06:17:00
2,2,50,M,False,2022-12-28 18:33:00,2022-12-29 03:20:00,2023-01-06 08:01:00,2022-12-28 18:47:00,NaN,2023-01-01 08:51:00,2023-01-02 22:15:00,2023-01-02 22:15:00,NaN,NaN,NaN,2023-01-06 07:42:00,2023-01-01 04:57:00,NaN,2022-12-29 22:37:00
3,3,63,M,False,2023-05-28 11:08:00,2023-05-28 13:35:00,2023-06-26 06:38:00,2023-05-28 11:21:00,NaN,NaN,NaN,2023-06-24 18:28:00,2023-05-29 05:56:00,2023-06-07 13:40:00,2023-06-01 08:31:00,2023-06-07 23:12:00,2023-05-30 11:21:00,NaN,2023-05-29 11:59:00
4,4,64,M,False,2022-01-21 19:48:00,2022-01-22 05:16:00,2022-02-01 20:01:00,2022-01-21 20:01:00,NaN,NaN,NaN,2022-01-22 06:43:00,2022-02-01 19:30:00,NaN,NaN,NaN,2022-01-22 06:57:00,NaN,2022-01-26 21:26:00


In [4]:
discharge_time = df['Discharge Time']
discharge_time = pd.to_datetime(discharge_time)

admission_time = df['Admission Time']
admission_time = pd.to_datetime(admission_time)

df['Admit Duration'] = discharge_time - admission_time

In [5]:
emergency_time = df['Emergency Dept Time']
emergency_time = pd.to_datetime(emergency_time)

ct_time = df['CT Scan Time']
ct_time = pd.to_datetime(ct_time)

df['Emergency Response Time'] = ct_time - emergency_time

## Demographics and Analysis with Comorbodities

In [6]:
@interact(Gender=widgets.Select(options=['All', 'Female', 'Male'], value='All'), Factor=['Age','Emergency Response Time', 'Admit Duration'], Show_Comorbidities=True)
def plot_demographic(Gender, Factor, Show_Comorbidities):

    if Gender=='Female':
        df_demo = df[df['Gender'] == 'F']
    elif Gender =='Male':
        df_demo =df[df['Gender'] == 'M']
    else:
        df_demo = df
    
    df_com = df_demo[df_demo['Comorbidities']==True]
    df_com_not = df_demo[df_demo['Comorbidities']==False]

    fig, ax = plt.subplots(figsize=(10,6))

    if Show_Comorbidities:
        if Factor=='Age':
            sns.histplot(df_com[Factor], label='Comorbidities', element='step', stat='frequency',ec='black', color=RED) 
            sns.histplot(df_com_not[Factor], label='Non-Comorbidities', element='step', stat='frequency',ec='black', color=BLUE)
            ax.set_xlabel('Age (years)') 
        else:
            if Factor == 'Admit Duration':
                sns.histplot(df_com[Factor].dt.total_seconds() / 86400, label='Comorbidities', element='step',ec='black', stat='frequency', color=RED)
                sns.histplot(df_com_not[Factor].dt.total_seconds() / 86400, label='Non-Comorbidities', element='step',ec='black', stat='frequency', color=BLUE)
                ax.set_xlabel('Admit Time (days)')
            else:
                sns.histplot(df_com[Factor].dt.total_seconds() / 60, label='Comorbidities', element='step',ec='black', stat='frequency', color=RED)
                sns.histplot(df_com_not[Factor].dt.total_seconds() / 60, label='Non-Comorbidities', element='step',ec='black', stat='frequency', color=BLUE)
                ax.set_xlabel('Response Time (minutes)')
    else:
        if Factor=='Age':
            sns.histplot(df_demo[Factor], label=Factor, element='step', color=OCHRE)
            ax.set_xlabel('Age (years)') 
        else:
            if Factor == 'Admit Duration':
                sns.histplot(df_demo[Factor].dt.total_seconds() / 86400, label=Factor, element='step',ec='black', stat='frequency', color=OCHRE)
                ax.set_xlabel('Admit Time (days)')
            else:
                sns.histplot(df_demo[Factor].dt.total_seconds() / 60, label=Factor, element='step',ec='black', stat='frequency', color=OCHRE)
                ax.set_xlabel('Response Time (minutes)')
        
    ax.legend()
    ax.set_ylabel('Patient Frequency')
    ax.set_title(f'{Gender} Demographic Data ({Factor})')
    plt.show()
    

interactive(children=(Select(description='Gender', options=('All', 'Female', 'Male'), value='All'), Dropdown(d…

## Attributions

- Widget List - https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
- Time Series / date functionality - https://pandas.pydata.org/docs/user_guide/timeseries.html
- Color Inspiration - https://crsn.ca/en/stroke-best-practices/best-practice-guidelines